## 9.4 双向循环神经网络
在序列学习中，我们以往假设的目标是：在给定观测的情况下(例如，在时间序列的上下文中或在语言模型的上下文中)，对下一个输出进行建模。虽然这是一个典型情景，但不是唯一。还可能发生什么其他的情况呢？我们考虑以下三个在文本序列中天空的任务：
- 我_ _ _。
- 我_ _ _ 饿了。
- 我_ _ _ 饿了，我可以吃半头猪。

根据可获得的信息量，我们可以用不同的词填空，如“很高兴(happy)”、“不(not)”和“非常(very)”。很明显，每个短语的“下文”传达了重要信息(如果有用的话)，而这些信息关乎到选择哪个词来填空，所以无法利用这一点的序列模型将在相关任务上表现不佳。例如，如果要做好命名实体识别(例如，识别“Green”指的是“格林先生”还是绿色)，不同长度的上下文范围重要性是相同的。为了获得一些解决问题的灵感，让我们先迂回到概率图模型

### 9.4.1 隐马尔可夫模型中的动态规划
这一小节是用来说明动态规划问题的，具体的技术细节对于理解深度学习模型并不重要，但它有助于我们思考为什么要使用深度学习，以及为什么要选择特定的架构。

如果我们想用概率图模型来解决这个问题，可以设计一个隐变量模型：在任意时间步t，假设存在某个隐变量$h_t$，通过概率$P(x_t \mid h_t)$控制我们观测到的$x_t$。此外，任何$h_t \rightarrow h_{t+1}$转移都是由一些状态转移概率$P(H_{t+1} \mid h_t)$给出。这个概率图模型就是一个隐马尔可夫模型(hidden Markov model, HMM)，如图9.4.1所示

<div align=center>
<img src='../../pics/9_4_1.jpeg' width='50%'>
</div>

因此，对于有T个观测值的序列，我们在观测状态和隐状态上具有以下联合概率分布：
$$
P(x_1, ..., x_T, h_1, ..., h_T) = \Pi_{t=1}^T P(h_t \mid h_{t-1}) P(x_t \mid h_t), where P(h_1 \mid h_0) = P(h_1)
\tag{9.4.1}
$$

现在，假设我们观测到所有的$x_i$，除了$x_j$，并且我们的目标是计算$P(x_j \mid x_{-j})$，其中$x_{-j}=(x_1, ..., x_{j-1}, x_{j+1}, ..., x_T)$。由于$P(x_j \mid x_{-j})$中没有隐变量，因此我们考虑对$h_1, ..., h_T$选择构成的所有可能的组合进行求和。如果任何$h_i$可以接受k个不同的值(有限的状态数)，这意味着我们需要对$k^T$个项求和，这个任务显然难于登天。幸运的是，有个巧妙的解决方案：动态规划(dynamic programming)。

要了解动态规划的工作方式，我们考虑对隐变量$h_1, ..., h_T$的依次求和。根据(9.4.1)，将得出：
$$
\begin{align*}
P(x_1, ..., x_T)\\
& = \sum_{h_1, ..., h_T} P(x_1, ..., x_T, h_1, ..., h_T)\\
& = \sum_{h_1, ..., h_T} \Pi_{t=1}^{T} P(h_t \mid h_{t-1}) P(x_t \mid h_t)\\
& = \sum_{h_2, ..., h_T} \underbrace{[\sum_{h_1} P(h_1) P(x_1 \mid h_1) P(h_2 \mid h_1)]}_{\pi_2(h_2) \overset{def}{=}} P(x_2 \mid h_2) \Pi_{t=3}^{T} P(h_t \mid h_{t-1}) P(x_t \mid h_t)\\
& = \sum_{h_3, ..., h_T} \underbrace{[\sum_{h_2} P(h_2) P(x_2 \mid h_2) P(h_3 \mid h_2)]}_{\pi_3(h_3) \overset{def}{=}} P(x_3 \mid h_3) \Pi_{t=4}^{T} P(h_t \mid h_{t-1}) P(x_t \mid h_t)\\
& = ... \\
& = \sum_{h_T} \pi_t(h_T)P(x_T \mid h_T)
\end{align*}
\tag{9.4.2}
$$

通常，我们将“前向递归”(forward recursion)写出：
$$
\pi_{t+1} (h_{t+1}) = \sum_{h_t} \pi_t(h_t)P(x_t \mid h_t) P(h_{t+1} \mid h_t)
\tag{9.4.3}
$$
递归被初始化为$\pi_1 (h_1) = P(h_1)$。符号简化，也可以写成$p_{t+1} = f(\pi_t, x_t)$，其中$f$是一些可学习的函数。这看起来就像我们在循环神经网络中讨论的隐变量模型的更新方程。

与前向递归一样，我们也可以使用后向递归对同一组隐变量求和。这将得到：
$$
\begin{align*}
P(x_1,..., x_T)\\
& = \sum_{h_1, ..., h_T} P(x_1, ..., x_T, h_1, ..., h_T)\\
& = \sum_{h_1, ..., h_T} \Pi_{t=1}^{T-1} P(h_t \mid h_{t-1}) P(x_t \mid h_t) · \underbrace{[\sum_{h_T} P(h_T \mid h_{T-1}) P(x_T \mid h_T)]}_{\rho_{T-1} (h_{T-1}) \overset{def}{=}}\\
& = \sum_{h_1, ..., h_T} \Pi_{t=1}^{T-2} P(h_t \mid h_{t-1}) P(x_t \mid h_t) · \underbrace{[\sum_{h_T-1} P(h_{T-1} \mid h_{T-2}) P(x_{T-1} \mid h_{T-1}) \rho_{T-1}(h_{T-1})]}_{\rho_{T-2} (h_{T-2}) \overset{def}{=}}\\
& = ...\\
& = \sum_{h_1} P(h_1) P(x_1 \mid h_1) \rho_1 (h_1)
\end{align*}
\tag{9.4.4}
$$
因此，我们可以将“后向递归”(backward recursion)写为：
$$
\rho_{t-1}(h_{t-1}) = \sum_{h_t} P(h_t \mid h_{t-1}) P(x_t \mid h_t) \rho_t (h_t)
\tag{9.4.5}
$$

初始化$\rho_T(h_T)=1$。前向和后向递归都允许我们对T个隐变量在$O(kT)$(线性而不是指数)时间内对($h_1, ..., h_T$)的所有值求和。这是使用图模型进行概率推理的巨大好处之一。它也是通用消息传递算法的一个非常特殊的例子。结合前向和后向递归，我们能够计算
$$
P(x_j\mid x_{-j}) \propto \sum_{h_j} \pi_j(h_j) \rho_{h_j} P(x_j \mid h_j)
\tag{9.4.6}
$$

因为符号简化的需要，后向递归也可以写为$\rho_{t-1} = g(\rho_t, x_t)$，其中$g$是一个可以学习的函数。同样，这看起来非常像一个更新方程，只是不像我们在循环神经网络中看到的那样前向运算，而是后向计算。事实上，知道未来数据何时可用对隐马尔可夫模型是有益的。信号处理学家将是知道未来观测这两种情况区分内插和外推。

### 9.4.2 双向模型
如果我们希望在循环神经网络中拥有一种机制，使之能够提供与隐马尔可夫模型类似的前瞻能力，我们就需要修改循环神经网络的设计。幸运的是，这在概念上很容易，只需要增加一个“从最后一个词元开始从后向前运行”的循环神经网络，而不是只有一个前向模式下“从第一个词元开始运行的”循环神经网络。双向循环神经网络(bidirectional RNNs)添加了反向传递信息的隐藏层，以便更灵活地处理此类信息。图9.4.2描述了具有单个隐藏层的双向循环神经网络的架构。

<div align=center>
<img src='../../pics/9_4_2.jpeg' width='50%'>
</div>

事实上，这与隐马尔可夫模型中的动态规划的前向和后向递归没有太大区别。其主要区别是，在隐马尔可夫模型中的方程具有特定的统计意义。双向循环神经网络没有这样容易理解的解释，我们只能把它们当作通用的、可学习的函数。这种转变集中体现了现代深度网络的设计原则：首先使用经典统计模型的函数依赖类型，然后将其参数化为通用形式。

**定义**

双向循环神经网络是由[Schuster & Paliwal, 1997]提出的。对于任意时间步t，给定一个小批量的输入数据$X_t \in \mathbb R ^{n \times d}$(样本数：n，每个示例中的输入数：d)，并且令隐藏层激活函数为$\phi$。在双向架构中，我们设该时间步的前向和反向隐状态分别为$\overrightarrow{H_t} \in \mathbb R^{n \times h}$和$\overleftarrow{H_t} \in \mathbb R^{n \times h}$，其中h是隐藏单元的数目。前向和反向隐状态的更新如下：
$$
\overrightarrow H_t = \phi (X_t W_{xh}^{(f)} + \overrightarrow H_{t-1} W_{hh}^{(f)} + b_h^{(f)})\\
\overleftarrow H_t = \phi (X_t W_{xh}^{(f)} + \overleftarrow H_{t+1} W_{hh}^{(f)} + b_h^{(b)})\\
\tag{9.4.7}
$$

其中，权重$W_{xh}^{(f)} \in \mathbb R^{d \times h}, W_{hh}^{(f)} \in \mathbb R^{h \times h}, W_{xh}^{(b)} \in \mathbb R^{d \times h}, W_{hh}^{(b)} \in \mathbb R^{h \times h}$和偏置$b_h^{(f)} \in \mathbb R^{1 \times h}, b_h^{(b)} \in \mathbb R^{1 \times h}$都是模型参数。